# Adding Dependeces

In [88]:
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/solvers.jl")

fusion (generic function with 1 method)

# Initial Conditions

In [89]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
T_f = 100000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 12

# Forces Conditions
R_Max = 2.6:0.2:3.0
Fp = 0.0:0.03:0.1
K_array = 0.005:0.006:0.032
s = 2.0

# Coding C0nditions
n_text = 50
num_agg = 2

2

In [90]:
# Testing all the conditions of the fusion
for t_f in T_f
    for r_max in R_Max
        for fp in Fp
            for K in K_array
                # Initial Data
                X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

                # Generating the Path
                if num_agg == 1
                    Path = "../../results/OneAgg/T_$(t_f)/fp_($(fp))/K_($(K)"
                else
                    Path = "../../results/TwoAgg/T_$(t_f)/fp_($(fp))/K_($(K))"
                end
                File = "tf_($(t_f))|s=($(s))|fp_($(fp))|K_($(K))|rmax_$(r_max)|_GPU.xyz"

                for p in 4:7 # p is the position every folder on the path
                    if !isdir(join(split(Path, "/")[1:p],"/")) 
                        mkdir(join(split(Path, "/")[1:p],"/")) 
                    end
                end

                # # Calculating all above
                println("---------------------------------------------------------------")
                if num_agg == 1
                    println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | fp = $(fp) | k = $(K) \n for $(num_agg) Aggregate")
                else
                    println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | fp = $(fp) | k = $(K) \n for $(num_agg) Aggregates")
                end
                if File in readdir(Path)
                    if countlines(Path*"/"*File) < (2*size(X,1)+2)*(n_text+1)
                        println("Calculated with less data. Recalculating")
                        rm(Path*"/"*File)
                        if num_agg == 1
                            one_aggregate(Path*"/"*File,true,n_text,t_f, r_max, fp, K)
                        else
                            fusion(Path*"/"*File,n_text,t_f, r_max, fp, K)
                        end
                    else
                        println("This is already calculated")           
                    end
                else
                    if num_agg == 1
                        one_aggregate(Path*"/"*File,true,n_text,t_f, r_max, fp, K)
                    else
                        fusion(Path*"/"*File,n_text,t_f, r_max, fp, K)
                    end
                end
            end
        end
    end
end

---------------------------------------------------------------
Calculating T_Final=100000 | R_Max = 3.5 | fp = 0.0 | k = 1.0e-5 
 for 2 Aggregates
Finding Equilibrium in one Aggregate


Progress: 100%|█████████████████████████| Time: 0:00:23


Finding Equilibrium in two Aggregate


Progress: 100%|█████████████████████████| Time: 0:05:54


---------------------------------------------------------------
Calculating T_Final=100000 | R_Max = 3.5 | fp = 0.0 | k = 0.0004 
 for 2 Aggregates
Finding Equilibrium in one Aggregate


...

Progress: 100%|█████████████████████████| Time: 0:05:55


---------------------------------------------------------------
Calculating T_Final=100000 | R_Max = 3.5 | fp = 0.0 | k = 0.05 
 for 2 Aggregates
Finding Equilibrium in one Aggregate


Progress: 100%|█████████████████████████| Time: 0:00:23


Finding Equilibrium in two Aggregate


Progress: 100%|█████████████████████████| Time: 0:05:53


In [91]:
f = collect(0.0:0.03:0.1)

4-element Vector{Float64}:
 0.0
 0.03
 0.06
 0.09

In [92]:
ka = collect(0.005:0.006:0.032)

5-element Vector{Float64}:
 0.005
 0.011
 0.017
 0.023
 0.029

In [93]:
collect(2.6:0.2:3.0)
# rmex = 

3-element Vector{Float64}:
 2.6
 2.8
 3.0